<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to Enforce HTTP Redirection Across AWS ALB unSkript legos.</b>
</div>

<br>

<center><h2>Enforce HTTP Redirection Across AWS ALB</h2></center>

# Steps Overview
    1) List All Application Load Balancer ARNs.
    2) Get all the Application Load Balancer Listener without HTTP Redirection.
    3) Modify the Application Load Balancer Listener for HTTP Redirection.

Here we will use unSkript AWS List Application LoadBalancers ARNs Lego. This lego take region: str as input. This input is used to list all of the Application Load Balancer ARNs.

In [2]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, List
import pprint


from beartype import beartype
@beartype
def aws_list_application_loadbalancers_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_application_loadbalancers(handle, region: str) -> List:
    """aws_list_application_loadbalancers lists application loadbalancers ARNs.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Region of the Classic loadbalancer.

        :rtype: List with all the application loadbalancer ARNs
    """

    ec2Client = handle.client('elbv2', region_name=region)
    resp = ec2Client.describe_load_balancers()
    result = []
    for elb in resp['LoadBalancers']:
        if elb['Type'] == "application":
            result.append(elb['LoadBalancerArn'])
    return result


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "Region"
    }''')
task.configure(outputName="LoadBalancerArn")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_application_loadbalancers, lego_printer=aws_list_application_loadbalancers_printer, hdl=hdl, args=args)

Here we will use unSkript Get AWS ALB Listeners Without HTTP Redirection Lego. This lego take loadbalancer_arn: list, region: str as input. This input is used to find out all Application Load Balancer listeners without HTTP Redirection.

In [3]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
import pprint


from beartype import beartype
@beartype
def aws_listeners_without_http_redirect_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_listeners_without_http_redirect(handle, loadbalancer_arn: list, region: str) -> List:
    """aws_get_auto_scaling_instances List of Dict with instanceId and attached groups.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type instance_ids: list
        :param instance_ids: List of LoadBalancerArn.

        :type region: string
        :param region: Region to filter ALB listeners.

        :rtype: List of ALB listeners without HTTP redirection.
    """
    ec2Client = handle.client('elbv2', region_name=region)
    result = []
    for alb in loadbalancer_arn:
        try:
            response = aws_get_paginator(ec2Client, "describe_listeners", "Listeners",
                                         LoadBalancerArn=alb)
            for listner in response:
                if 'SslPolicy' not in listner:
                    resp = aws_get_paginator(ec2Client, "describe_rules", "Rules",
                                         ListenerArn=listner['ListenerArn'])
                    for rule in resp:
                        for action in rule['Actions']:
                            if action['Type'] != 'redirect':
                                result.append(listner['ListenerArn'])
        except Exception as error:
            result.append(error)
    return result






task = Task(Workflow())
task.configure(inputParamsJson='''{
    "loadbalancer_arn": "LoadBalancerArn",
    "region": "Region"
    }''')
task.configure(outputName="ListenerARNs")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_listeners_without_http_redirect, lego_printer=aws_listeners_without_http_redirect_printer, hdl=hdl, args=args)

Here we will use unSkript AWS Modify ALB Listeners HTTP Redirection Lego. This lego take listener_arn: str, region: str as input. This input is used to modify Application Load Balancer listeners HTTP Redirection.

In [6]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint

from beartype import beartype
@beartype
def aws_modify_listener_for_http_redirection_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_modify_listener_for_http_redirection(handle, listener_arn: str, region: str) -> List:
    """aws_modify_listener_for_http_redirection List of Dict with modified listener info.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type listener_arn: string
        :param listener_arn: List of LoadBalancerArn.

        :type region: string
        :param region: Region to filter ALB listeners.

        :rtype: List of Dict with modified ALB listeners info.
    """
    listner_config = [{
                        "Type": "redirect",
                        "Order": 1,
                        "RedirectConfig": {
                            "Protocol": "HTTPS",
                            "Host": "#{host}",
                            "Query": "#{query}",
                            "Path": "/#{path}",
                            "Port": "443",
                            "StatusCode": "HTTP_302"}}]
    result = []
    try:
        #if ALB_Name in listener_arn:
        ec2Client = handle.client('elbv2', region_name=region)
        response = ec2Client.modify_listener(ListenerArn=listener_arn,
                                                 DefaultActions=listner_config)
            result.append(response)

    except Exception as error:
        result.append(error)

    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "listener_arn": "iter_item",
    "region": "Region"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "ListenerARNs",
    "iter_parameter": "listener_arn"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(ListenerARNs)>0",
    "condition_result": true
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_modify_listener_for_http_redirection, lego_printer=aws_modify_listener_for_http_redirection_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS legos to perform AWS actions and this runbook find aut all the Application Load Balancer listeners without HTTP redirection and midify them for HTTP redirection. To view the full platform capabilities of unSkript please visit https://unskript.com